<a href="https://www.kaggle.com/code/khoatran311/traffic-volume-data-processing?scriptVersionId=221152849" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as pls
import seaborn as sea
import matplotlib.pyplot as plt
import datetime
import scipy.stats as stats
from scipy.interpolate import CubicSpline

import warnings
warnings.filterwarnings("ignore")

# Loading Data

In [2]:
def format_pl():
    """FLOAT DISPLAY FORMATTING"""
    pl.Config.set_fmt_float("mixed")
    """STRING FORMATTING"""
    pl.Config.set_fmt_str_lengths(50)
    """TABLE FORMATTING"""
    pl.Config.set_tbl_rows(8)
    pl.Config.set_tbl_cols(15)
    pl.Config.set_tbl_width_chars(200)
    pl.Config.set_tbl_cell_alignment("RIGHT")
    pl.Config.set_tbl_hide_dtype_separator(True)
    pl.Config.set_tbl_hide_column_data_types(True)

format_pl()

In [3]:
data = pl.read_csv("/kaggle/input/metro-interstate-traffic-volume/Metro_Interstate_Traffic_Volume.csv")

data

holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
"""None""",288.28,0.0,0.0,40,"""Clouds""","""scattered clouds""","""2012-10-02 09:00:00""",5545
"""None""",289.36,0.0,0.0,75,"""Clouds""","""broken clouds""","""2012-10-02 10:00:00""",4516
"""None""",289.58,0.0,0.0,90,"""Clouds""","""overcast clouds""","""2012-10-02 11:00:00""",4767
"""None""",290.13,0.0,0.0,90,"""Clouds""","""overcast clouds""","""2012-10-02 12:00:00""",5026
…,…,…,…,…,…,…,…,…
"""None""",282.76,0.0,0.0,90,"""Clouds""","""overcast clouds""","""2018-09-30 20:00:00""",2781
"""None""",282.73,0.0,0.0,90,"""Thunderstorm""","""proximity thunderstorm""","""2018-09-30 21:00:00""",2159
"""None""",282.09,0.0,0.0,90,"""Clouds""","""overcast clouds""","""2018-09-30 22:00:00""",1450
"""None""",282.12,0.0,0.0,90,"""Clouds""","""overcast clouds""","""2018-09-30 23:00:00""",954


In [4]:
for feature,dtype in zip(data.columns, data.dtypes):
    print(f"Feature: {feature}")
    print(f"Data Type: {dtype}\n")

Feature: holiday
Data Type: String

Feature: temp
Data Type: Float64

Feature: rain_1h
Data Type: Float64

Feature: snow_1h
Data Type: Float64

Feature: clouds_all
Data Type: Int64

Feature: weather_main
Data Type: String

Feature: weather_description
Data Type: String

Feature: date_time
Data Type: String

Feature: traffic_volume
Data Type: Int64



In [5]:
data = data.with_columns(
    pl.col("date_time").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("date_time")
).rename(
    {
        "temp":"temp_kelvin",
        "rain_1h":"rain_1h_mm",
        "snow_1h":"snow_1h_mm",
        "clouds_all":"cloud_pct",
        "weather_description":"weather_desc",
        "date_time":"datetime"
    }
)

data

holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,datetime,traffic_volume
"""None""",288.28,0.0,0.0,40,"""Clouds""","""scattered clouds""",2012-10-02 09:00:00,5545
"""None""",289.36,0.0,0.0,75,"""Clouds""","""broken clouds""",2012-10-02 10:00:00,4516
"""None""",289.58,0.0,0.0,90,"""Clouds""","""overcast clouds""",2012-10-02 11:00:00,4767
"""None""",290.13,0.0,0.0,90,"""Clouds""","""overcast clouds""",2012-10-02 12:00:00,5026
…,…,…,…,…,…,…,…,…
"""None""",282.76,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 20:00:00,2781
"""None""",282.73,0.0,0.0,90,"""Thunderstorm""","""proximity thunderstorm""",2018-09-30 21:00:00,2159
"""None""",282.09,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 22:00:00,1450
"""None""",282.12,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 23:00:00,954


In [6]:
data.select(
    pl.all().is_null().sum()
).unpivot(
    value_name="NA_count"
).sort(by="NA_count", descending=True)

variable,NA_count
"""holiday""",0
"""temp_kelvin""",0
"""rain_1h_mm""",0
"""snow_1h_mm""",0
…,…
"""weather_main""",0
"""weather_desc""",0
"""datetime""",0
"""traffic_volume""",0


In [7]:
data.select(
    pl.all().n_unique().sum()
).unpivot(
    value_name="unique_count"
).sort(by="unique_count", descending=True)

variable,unique_count
"""datetime""",40575
"""traffic_volume""",6704
"""temp_kelvin""",5843
"""rain_1h_mm""",372
…,…
"""weather_desc""",38
"""holiday""",12
"""snow_1h_mm""",12
"""weather_main""",11


## Categorical Data Analysis
### 1. Created category "rare weather" for **weather_desc** values occurring less than 50 times
### 2. Concatenated **weather_main** and **weather_desc** together, creating a new feature

In [8]:
cat_data = data.select(pl.col(pl.Utf8))

In [9]:
for feature in cat_data.columns:
    df = data.group_by(feature).agg(
    pl.col(feature).count().alias("count"),
    ).with_columns(
        (pl.col("count")/pl.col("count").sum()).alias("prop")
    ).sort(by="count", descending=True)
    print(df)
    print()

shape: (12, 3)
┌──────────────────┬───────┬──────────┐
│          holiday ┆ count ┆     prop │
╞══════════════════╪═══════╪══════════╡
│             None ┆ 48143 ┆ 0.998735 │
│        Labor Day ┆     7 ┆ 0.000145 │
│    Christmas Day ┆     6 ┆ 0.000124 │
│    New Years Day ┆     6 ┆ 0.000124 │
│                … ┆     … ┆        … │
│     Memorial Day ┆     5 ┆ 0.000104 │
│     Veterans Day ┆     5 ┆ 0.000104 │
│ Independence Day ┆     5 ┆ 0.000104 │
│       State Fair ┆     5 ┆ 0.000104 │
└──────────────────┴───────┴──────────┘

shape: (11, 3)
┌──────────────┬───────┬──────────┐
│ weather_main ┆ count ┆     prop │
╞══════════════╪═══════╪══════════╡
│       Clouds ┆ 15164 ┆  0.31458 │
│        Clear ┆ 13391 ┆ 0.277799 │
│         Mist ┆  5950 ┆ 0.123434 │
│         Rain ┆  5672 ┆ 0.117667 │
│            … ┆     … ┆        … │
│ Thunderstorm ┆  1034 ┆ 0.021451 │
│          Fog ┆   912 ┆  0.01892 │
│        Smoke ┆    20 ┆ 0.000415 │
│       Squall ┆     4 ┆ 0.000083 │
└──────────────┴─

In [10]:
df = data.group_by(feature).agg(
pl.col("weather_desc").count().alias("count"),
).with_columns(
    (pl.col("count")/pl.col("count").sum()).alias("prop")
).sort(by="count", descending=True)

for desc,count,prop in zip(df["weather_desc"].to_list(), df["count"].to_list(), df["prop"].to_list()):
    print(f"Description:  {desc}")
    print(f"Count:        {count}")
    print(f"Proportion:   {prop:.3f}\n")

Description:  sky is clear
Count:        11665
Proportion:   0.242

Description:  mist
Count:        5950
Proportion:   0.123

Description:  overcast clouds
Count:        5081
Proportion:   0.105

Description:  broken clouds
Count:        4666
Proportion:   0.097

Description:  scattered clouds
Count:        3461
Proportion:   0.072

Description:  light rain
Count:        3372
Proportion:   0.070

Description:  few clouds
Count:        1956
Proportion:   0.041

Description:  light snow
Count:        1946
Proportion:   0.040

Description:  Sky is Clear
Count:        1726
Proportion:   0.036

Description:  moderate rain
Count:        1664
Proportion:   0.035

Description:  haze
Count:        1360
Proportion:   0.028

Description:  light intensity drizzle
Count:        1100
Proportion:   0.023

Description:  fog
Count:        912
Proportion:   0.019

Description:  proximity thunderstorm
Count:        673
Proportion:   0.014

Description:  drizzle
Count:        651
Proportion:   0.014

Des

In [11]:
### Obtain values with count < 50
rare_desc = set([
    desc for desc,count in zip(df["weather_desc"].to_list(), df["count"].to_list()) 
    if count < 50
])

### Encode any values in rare_desc as "rare weather"
new_data = data.with_columns(
    pl.when(pl.col("weather_desc").is_in(rare_desc))
    .then(pl.lit("rare weather"))
    .otherwise(pl.col("weather_desc"))
    .alias("weather_desc")
)


## Combine weather description columns together
new_data = new_data.with_columns(
    (pl.col("weather_main") + "__" + pl.col("weather_desc")).alias("weather_main_desc_combined")
)

In [12]:
df = new_data.group_by(
    pl.col("weather_main_desc_combined")
).agg(
    pl.col("weather_main_desc_combined").count().alias("count")
).sort(
    by="count", descending=True
)

for desc,count in zip(df["weather_main_desc_combined"].to_list(), 
                      df["count"].to_list()):
    print(f"Description:  {desc}")
    print(f"Count:        {count}")

Description:  Clear__sky is clear
Count:        11665
Description:  Mist__mist
Count:        5950
Description:  Clouds__overcast clouds
Count:        5081
Description:  Clouds__broken clouds
Count:        4666
Description:  Clouds__scattered clouds
Count:        3461
Description:  Rain__light rain
Count:        3372
Description:  Clouds__few clouds
Count:        1956
Description:  Snow__light snow
Count:        1946
Description:  Clear__Sky is Clear
Count:        1726
Description:  Rain__moderate rain
Count:        1664
Description:  Haze__haze
Count:        1360
Description:  Drizzle__light intensity drizzle
Count:        1100
Description:  Fog__fog
Count:        912
Description:  Thunderstorm__proximity thunderstorm
Count:        673
Description:  Drizzle__drizzle
Count:        651
Description:  Snow__heavy snow
Count:        616
Description:  Rain__heavy intensity rain
Count:        467
Description:  Snow__snow
Count:        293
Description:  Rain__proximity shower rain
Count:      

## Numerical Data Anlysis
### 1. Resampled the data to follow a consistent 1-hour interval
### 2. Filtered only data after 2016/1/1 to use (earlier data contain large gaps of missing values)
### 3. Interpolated missing numeric values with Cubic Spline
### 4. Interpolated missing categorical values with forward and backward filling

In [13]:
new_data.with_columns(
    pl.col("datetime").diff().alias("diff_datetime")
).group_by(
    pl.col("diff_datetime")
).agg(
    pl.col("diff_datetime").count().alias("count")
).sort(by="diff_datetime", descending=True).to_pandas()

,diff_datetime,count
0,NaT,0
1,307 days 19:00:00,1
2,10 days 03:00:00,1
3,4 days 22:00:00,1
4,4 days 21:00:00,1
5,3 days 21:00:00,1
6,2 days 17:00:00,1
7,1 days 23:00:00,1
8,1 days 10:00:00,1
9,1 days 09:00:00,1


In [14]:
new_data.with_columns(
    pl.col("datetime").diff().alias("diff_datetime")
).filter(
    pl.col("diff_datetime")==datetime.timedelta(days=307, seconds=68400)
)

holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,datetime,traffic_volume,weather_main_desc_combined,diff_datetime
"""None""",287.586,0.0,0.0,92,"""Rain""","""light rain""",2015-06-11 20:00:00,2886,"""Rain__light rain""",307d 19h


In [15]:
"""Filters the data to contain dates after 2015-06-12 only (eliminates the large break in the datetime interval)"""
new_data = new_data.filter(
    pl.col("datetime") >= datetime.datetime(2015, 6, 12, 0)
)

new_data

holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,datetime,traffic_volume,weather_main_desc_combined
"""None""",297.543,0.0,0.0,0,"""Clear""","""Sky is Clear""",2015-06-12 18:00:00,4380,"""Clear__Sky is Clear"""
"""None""",292.061,0.0,0.0,8,"""Clear""","""sky is clear""",2015-06-12 20:00:00,3095,"""Clear__sky is clear"""
"""None""",292.061,0.0,0.0,8,"""Clear""","""sky is clear""",2015-06-12 21:00:00,3299,"""Clear__sky is clear"""
"""None""",294.84,0.0,0.0,76,"""Clouds""","""broken clouds""",2015-06-13 21:00:00,3040,"""Clouds__broken clouds"""
…,…,…,…,…,…,…,…,…,…
"""None""",282.76,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 20:00:00,2781,"""Clouds__overcast clouds"""
"""None""",282.73,0.0,0.0,90,"""Thunderstorm""","""proximity thunderstorm""",2018-09-30 21:00:00,2159,"""Thunderstorm__proximity thunderstorm"""
"""None""",282.09,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 22:00:00,1450,"""Clouds__overcast clouds"""
"""None""",282.12,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 23:00:00,954,"""Clouds__overcast clouds"""


In [16]:
"""Inspects uniformity in sampling interval for values"""

new_data.with_columns(
    pl.col("datetime").diff().alias("diff_datetime")
).group_by(
    pl.col("diff_datetime")
).agg(
    pl.col("diff_datetime").count().alias("count")
).sort(by="diff_datetime", descending=True).to_pandas()

,diff_datetime,count
0,NaT,0
1,4 days 22:00:00,1
2,3 days 21:00:00,1
3,1 days 10:00:00,1
4,1 days 06:00:00,1
5,1 days 00:00:00,2
6,0 days 21:00:00,1
7,0 days 18:00:00,1
8,0 days 14:00:00,1
9,0 days 10:00:00,4


In [17]:
"""
1. Remove data entries with 0 changes in datetime (possible duplicates)
2. Drops diff_datetime feature
"""
new_data = new_data.with_columns(
    pl.col("datetime").diff().alias("diff_datetime")
).filter(
    pl.col("diff_datetime")!=0
).drop("diff_datetime")

new_data

holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,datetime,traffic_volume,weather_main_desc_combined
"""None""",292.061,0.0,0.0,8,"""Clear""","""sky is clear""",2015-06-12 20:00:00,3095,"""Clear__sky is clear"""
"""None""",292.061,0.0,0.0,8,"""Clear""","""sky is clear""",2015-06-12 21:00:00,3299,"""Clear__sky is clear"""
"""None""",294.84,0.0,0.0,76,"""Clouds""","""broken clouds""",2015-06-13 21:00:00,3040,"""Clouds__broken clouds"""
"""None""",293.137,0.0,0.0,56,"""Clouds""","""broken clouds""",2015-06-13 23:00:00,2051,"""Clouds__broken clouds"""
…,…,…,…,…,…,…,…,…,…
"""None""",282.76,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 20:00:00,2781,"""Clouds__overcast clouds"""
"""None""",282.73,0.0,0.0,90,"""Thunderstorm""","""proximity thunderstorm""",2018-09-30 21:00:00,2159,"""Thunderstorm__proximity thunderstorm"""
"""None""",282.09,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 22:00:00,1450,"""Clouds__overcast clouds"""
"""None""",282.12,0.0,0.0,90,"""Clouds""","""overcast clouds""",2018-09-30 23:00:00,954,"""Clouds__overcast clouds"""


In [18]:
"""Resamples the data to contain uniform 1hr sampling rate"""

min_time, max_time = new_data["datetime"].min(), new_data["datetime"].max()

full_range = pl.DataFrame(
    {"datetime":pl.datetime_range(min_time, max_time, "1h", eager=True)}
)

resampled_data = full_range.join(new_data, on="datetime", how="left")
resampled_data

datetime,holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,traffic_volume,weather_main_desc_combined
2015-06-12 20:00:00,"""None""",292.061,0.0,0.0,8,"""Clear""","""sky is clear""",3095,"""Clear__sky is clear"""
2015-06-12 21:00:00,"""None""",292.061,0.0,0.0,8,"""Clear""","""sky is clear""",3299,"""Clear__sky is clear"""
2015-06-12 22:00:00,null,null,null,null,null,null,null,null,null
2015-06-12 23:00:00,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…
2018-09-30 20:00:00,"""None""",282.76,0.0,0.0,90,"""Clouds""","""overcast clouds""",2781,"""Clouds__overcast clouds"""
2018-09-30 21:00:00,"""None""",282.73,0.0,0.0,90,"""Thunderstorm""","""proximity thunderstorm""",2159,"""Thunderstorm__proximity thunderstorm"""
2018-09-30 22:00:00,"""None""",282.09,0.0,0.0,90,"""Clouds""","""overcast clouds""",1450,"""Clouds__overcast clouds"""
2018-09-30 23:00:00,"""None""",282.12,0.0,0.0,90,"""Clouds""","""overcast clouds""",954,"""Clouds__overcast clouds"""


In [19]:
"""
Finds the count of NA values in each feature per each year-month-day
1. The max number of missing values per datetime is 24
2. The datetime with the most missing values after resampling is 2015-06 and 2015-10 data
"""
NA_counts = resampled_data.with_columns(
    pl.col("datetime").dt.strftime("%Y-%m-%d").alias("year_month_day")
).with_columns(
    pl.col("year_month_day").str.strptime(pl.Datetime, "%Y-%m-%d")
).group_by(
    pl.col("year_month_day")
).agg(
    pl.all().is_null().sum()
).sort(
    by="year_month_day",
    descending=False
).filter(
    pl.col("holiday")>0
)

high_NA_counts = NA_counts.filter(
    pl.col("holiday")==24
).to_pandas()


display(NA_counts)
display(high_NA_counts)

year_month_day,datetime,holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,traffic_volume,weather_main_desc_combined
2015-06-12 00:00:00,0,2,2,2,2,2,2,2,2,2
2015-06-13 00:00:00,0,22,22,22,22,22,22,22,22,22
2015-06-14 00:00:00,0,23,23,23,23,23,23,23,23,23
2015-06-15 00:00:00,0,24,24,24,24,24,24,24,24,24
…,…,…,…,…,…,…,…,…,…,…
2018-05-05 00:00:00,0,1,1,1,1,1,1,1,1,1
2018-06-02 00:00:00,0,1,1,1,1,1,1,1,1,1
2018-08-07 00:00:00,0,3,3,3,3,3,3,3,3,3
2018-08-23 00:00:00,0,1,1,1,1,1,1,1,1,1


,year_month_day,datetime,holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,traffic_volume,weather_main_desc_combined
0,2015-06-15,0,24,24,24,24,24,24,24,24,24
1,2015-06-16,0,24,24,24,24,24,24,24,24,24
2,2015-06-17,0,24,24,24,24,24,24,24,24,24
3,2015-06-18,0,24,24,24,24,24,24,24,24,24
4,2015-06-21,0,24,24,24,24,24,24,24,24,24
5,2015-06-23,0,24,24,24,24,24,24,24,24,24
6,2015-10-24,0,24,24,24,24,24,24,24,24,24
7,2015-10-25,0,24,24,24,24,24,24,24,24,24
8,2015-10-26,0,24,24,24,24,24,24,24,24,24


In [20]:
"""
Takes data from 2016-1-1 and onward only, as data in 2015 is packed with missing values
"""
resampled_data = resampled_data.with_columns(
    pl.col("datetime").dt.strftime("%Y-%m-%d").alias("year_month_day")
).with_columns(
    pl.col("year_month_day").str.strptime(pl.Datetime, "%Y-%m-%d")
).filter(
    pl.col("year_month_day")>=datetime.datetime(2016, 1, 1)
).drop(
    pl.col("year_month_day")
)

resampled_data

datetime,holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,traffic_volume,weather_main_desc_combined
2016-01-01 00:00:00,"""New Years Day""",265.94,0.0,0.0,90,"""Haze""","""haze""",1513,"""Haze__haze"""
2016-01-01 01:00:00,"""None""",266.0,0.0,0.0,90,"""Snow""","""light snow""",1550,"""Snow__light snow"""
2016-01-01 02:00:00,null,null,null,null,null,null,null,null,null
2016-01-01 03:00:00,"""None""",266.01,0.0,0.0,90,"""Snow""","""light snow""",719,"""Snow__light snow"""
…,…,…,…,…,…,…,…,…,…
2018-09-30 20:00:00,"""None""",282.76,0.0,0.0,90,"""Clouds""","""overcast clouds""",2781,"""Clouds__overcast clouds"""
2018-09-30 21:00:00,"""None""",282.73,0.0,0.0,90,"""Thunderstorm""","""proximity thunderstorm""",2159,"""Thunderstorm__proximity thunderstorm"""
2018-09-30 22:00:00,"""None""",282.09,0.0,0.0,90,"""Clouds""","""overcast clouds""",1450,"""Clouds__overcast clouds"""
2018-09-30 23:00:00,"""None""",282.12,0.0,0.0,90,"""Clouds""","""overcast clouds""",954,"""Clouds__overcast clouds"""


In [21]:
"""
Applies Cubic Spline Interpolation to NUMERIC VARIABLES only
"""

resampled_data = resampled_data.with_columns(
    pl.col("datetime").cast(pl.Int64).alias("numeric_datetime")
)
num_vars = ["temp_kelvin", "rain_1h_mm", "snow_1h_mm", "cloud_pct","traffic_volume"]


interpolated_features = {}
for feature in num_vars:
    # Selects time and target features
    time = resampled_data["numeric_datetime"].to_numpy()  
    target = resampled_data[feature].to_numpy()  
    # Creates mask for missing values
    mask = ~np.isnan(target)
    # Applies cubic splinee
    spline = CubicSpline(time[mask], target[mask], extrapolate=True)
    target_filled = np.where(np.isnan(target), spline(time), target)
    target_filled = np.round(target_filled, 2)
    # Stores the interpolated feature
    interpolated_features[feature] = pl.Series(feature, target_filled)
    
# Replace features with interpolated features
resampled_data = resampled_data.with_columns(
    *interpolated_features.values()
)
resampled_data = resampled_data.drop("numeric_datetime")

resampled_data

datetime,holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,traffic_volume,weather_main_desc_combined
2016-01-01 00:00:00,"""New Years Day""",265.94,0.0,0.0,90.0,"""Haze""","""haze""",1513.0,"""Haze__haze"""
2016-01-01 01:00:00,"""None""",266.0,0.0,0.0,90.0,"""Snow""","""light snow""",1550.0,"""Snow__light snow"""
2016-01-01 02:00:00,null,266.34,-0.0,-0.0,91.72,null,null,1157.17,null
2016-01-01 03:00:00,"""None""",266.01,0.0,0.0,90.0,"""Snow""","""light snow""",719.0,"""Snow__light snow"""
…,…,…,…,…,…,…,…,…,…
2018-09-30 20:00:00,"""None""",282.76,0.0,0.0,90.0,"""Clouds""","""overcast clouds""",2781.0,"""Clouds__overcast clouds"""
2018-09-30 21:00:00,"""None""",282.73,0.0,0.0,90.0,"""Thunderstorm""","""proximity thunderstorm""",2159.0,"""Thunderstorm__proximity thunderstorm"""
2018-09-30 22:00:00,"""None""",282.09,0.0,0.0,90.0,"""Clouds""","""overcast clouds""",1450.0,"""Clouds__overcast clouds"""
2018-09-30 23:00:00,"""None""",282.12,0.0,0.0,90.0,"""Clouds""","""overcast clouds""",954.0,"""Clouds__overcast clouds"""


In [22]:
"""
Applies Forward and Backward filling for EXOGENOUS CATEGORICAL VARIABLES
"""
resampled_data = resampled_data.with_columns(
    [pl.col(feature).fill_null(strategy="forward")
     for feature in ["weather_main", "weather_desc", "weather_main_desc_combined"]
    ]  
).with_columns(
    pl.col("holiday").fill_null(strategy="backward").fill_null(strategy="forward")
)

resampled_data

datetime,holiday,temp_kelvin,rain_1h_mm,snow_1h_mm,cloud_pct,weather_main,weather_desc,traffic_volume,weather_main_desc_combined
2016-01-01 00:00:00,"""New Years Day""",265.94,0.0,0.0,90.0,"""Haze""","""haze""",1513.0,"""Haze__haze"""
2016-01-01 01:00:00,"""None""",266.0,0.0,0.0,90.0,"""Snow""","""light snow""",1550.0,"""Snow__light snow"""
2016-01-01 02:00:00,"""None""",266.34,-0.0,-0.0,91.72,"""Snow""","""light snow""",1157.17,"""Snow__light snow"""
2016-01-01 03:00:00,"""None""",266.01,0.0,0.0,90.0,"""Snow""","""light snow""",719.0,"""Snow__light snow"""
…,…,…,…,…,…,…,…,…,…
2018-09-30 20:00:00,"""None""",282.76,0.0,0.0,90.0,"""Clouds""","""overcast clouds""",2781.0,"""Clouds__overcast clouds"""
2018-09-30 21:00:00,"""None""",282.73,0.0,0.0,90.0,"""Thunderstorm""","""proximity thunderstorm""",2159.0,"""Thunderstorm__proximity thunderstorm"""
2018-09-30 22:00:00,"""None""",282.09,0.0,0.0,90.0,"""Clouds""","""overcast clouds""",1450.0,"""Clouds__overcast clouds"""
2018-09-30 23:00:00,"""None""",282.12,0.0,0.0,90.0,"""Clouds""","""overcast clouds""",954.0,"""Clouds__overcast clouds"""


# Saves Data File

In [23]:
resampled_data.write_csv("resampled_data.csv")